#Creating a synthetic Q&A dataset

In [1]:
!pip install --upgrade openai --quiet

from google.colab import userdata
OPEN_AI_KEY=userdata.get('opeaikey4o')

from openai import OpenAI
import matplotlib.pyplot as plt
import time

## Set the API key and model name
MODEL="gpt-3.5-turbo"
client = OpenAI(api_key=OPEN_AI_KEY)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00


In [5]:
import pandas as pd
df = pd.read_csv('olympics_sections_lg.csv')
df.head()

,title,heading,content,tokens
0,2020 Summer Olympics,Summary,"The 2020 Summer Olympics, officially the Games...",623
1,2016 Winter Youth Olympics,Summary,The 2016 Winter Youth Olympics (Norwegian: Oly...,152
2,2016 Winter Youth Olympics,Host selection,Lillehammer was the only city to bid for the g...,129
3,Olympic Games ceremony,Summary,The Olympic Games ceremonies of the Ancient Ol...,271
4,Olympic Games ceremony,Ancient forerunners,"The Ancient Games, held in Greece from ca. 776...",247


In [6]:
df['context'] = df.title + "\n" + df.heading + "\n\n" + df.content
df.head()

,title,heading,content,tokens,context
0,2020 Summer Olympics,Summary,"The 2020 Summer Olympics, officially the Games...",623,2020 Summer Olympics\nSummary\n\nThe 2020 Summ...
1,2016 Winter Youth Olympics,Summary,The 2016 Winter Youth Olympics (Norwegian: Oly...,152,2016 Winter Youth Olympics\nSummary\n\nThe 201...
2,2016 Winter Youth Olympics,Host selection,Lillehammer was the only city to bid for the g...,129,2016 Winter Youth Olympics\nHost selection\n\n...
3,Olympic Games ceremony,Summary,The Olympic Games ceremonies of the Ancient Ol...,271,Olympic Games ceremony\nSummary\n\nThe Olympic...
4,Olympic Games ceremony,Ancient forerunners,"The Ancient Games, held in Greece from ca. 776...",247,Olympic Games ceremony\nAncient forerunners\n\...


In [8]:
df['context'][0]

'2020 Summer Olympics\nSummary\n\nThe 2020 Summer Olympics, officially the Games of the XXXII Olympiad and officially branded as Tokyo 2020, were an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina on 7 September 2013.\nOriginally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. 

In [9]:
len(df)

759

#Create questions based on the context

Use gpt 3.5 turbo to generate a number of plausible questions relating to the Wikipedia section contents.

Note: We have used temperature=0, but it may be beneficial to experiment with a higher temperature to get a higher diversity of questions.

WARNING: This step will last a long time, and consume a lot of tokens, as it calls gpt-3.5 for every section to generate a number of questions.

In [10]:


def get_questions(context):
  try:
    qa_audio_response = client.chat.completions.create(
    model=MODEL,
    messages=[
    {"role": "system", "content":"""Use the context to generate   question. Respond in Markdown."""},
    {"role": "user", "content": f"The context is: {context}. \n\n"},
    ],
    temperature=0,
          )
    return qa_audio_response.choices[0].message.content
  except:
      return ""




In [11]:
df['questions']= df.context.apply(get_questions)


In [12]:
df.head()

,title,heading,content,tokens,context,questions
0,2020 Summer Olympics,Summary,"The 2020 Summer Olympics, officially the Games...",623,2020 Summer Olympics\nSummary\n\nThe 2020 Summ...,What were some of the new events introduced in...
1,2016 Winter Youth Olympics,Summary,The 2016 Winter Youth Olympics (Norwegian: Oly...,152,2016 Winter Youth Olympics\nSummary\n\nThe 201...,What was significant about the 2016 Winter You...
2,2016 Winter Youth Olympics,Host selection,Lillehammer was the only city to bid for the g...,129,2016 Winter Youth Olympics\nHost selection\n\n...,What was the process for selecting the host ci...
3,Olympic Games ceremony,Summary,The Olympic Games ceremonies of the Ancient Ol...,271,Olympic Games ceremony\nSummary\n\nThe Olympic...,What are some elements of the modern Olympic G...
4,Olympic Games ceremony,Ancient forerunners,"The Ancient Games, held in Greece from ca. 776...",247,Olympic Games ceremony\nAncient forerunners\n\...,What were some of the elements involved in the...


In [14]:
print(df[['context']].values[0][0])

2020 Summer Olympics
Summary

The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and officially branded as Tokyo 2020, were an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina on 7 September 2013.
Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The G

In [13]:
print(df[['questions']].values[0][0])

What were some of the new events introduced in existing sports at the 2020 Summer Olympics in Tokyo?


In [15]:
print(df.content.values[0])

The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and officially branded as Tokyo 2020, were an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina on 7 September 2013.
Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive e

In [ ]:
df.head()

#Create answers based on the context
WARNING: This step will last a long time, and consume a lot of tokens, as it calls davinci-instruct for every section to answer all the questions.

In [16]:
def get_answers(row):
  try:
    qa_audio_response = client.chat.completions.create(
    model=MODEL,
    messages=[
    {"role": "system", "content":"""Use the context and question  to generate   answer. Respond in Markdown."""},
    {"role": "user", "content": f"The context is: {row.context}. \n\nQuestions:\n{row.questions}\n\n"},
    ],
    temperature=0,
          )
    return qa_audio_response.choices[0].message.content
  except:
      return ""




In [17]:
df['answers']= df.apply(get_answers, axis=1)


In [18]:
df.head()

,title,heading,content,tokens,context,questions,answers
0,2020 Summer Olympics,Summary,"The 2020 Summer Olympics, officially the Games...",623,2020 Summer Olympics\nSummary\n\nThe 2020 Summ...,What were some of the new events introduced in...,Some of the new events introduced in existing ...
1,2016 Winter Youth Olympics,Summary,The 2016 Winter Youth Olympics (Norwegian: Oly...,152,2016 Winter Youth Olympics\nSummary\n\nThe 201...,What was significant about the 2016 Winter You...,The 2016 Winter Youth Olympics held in Lilleha...
2,2016 Winter Youth Olympics,Host selection,Lillehammer was the only city to bid for the g...,129,2016 Winter Youth Olympics\nHost selection\n\n...,What was the process for selecting the host ci...,The process for selecting the host city for th...
3,Olympic Games ceremony,Summary,The Olympic Games ceremonies of the Ancient Ol...,271,Olympic Games ceremony\nSummary\n\nThe Olympic...,What are some elements of the modern Olympic G...,Some elements of the modern Olympic Games cere...
4,Olympic Games ceremony,Ancient forerunners,"The Ancient Games, held in Greece from ca. 776...",247,Olympic Games ceremony\nAncient forerunners\n\...,What were some of the elements involved in the...,Some of the elements involved in the victory c...


In [22]:
len(df)

759

In [23]:
df = df.dropna().reset_index().drop('index',axis=1)
df.head()

,title,heading,content,tokens,context,questions,answers
0,2020 Summer Olympics,Summary,"The 2020 Summer Olympics, officially the Games...",623,2020 Summer Olympics\nSummary\n\nThe 2020 Summ...,What were some of the new events introduced in...,Some of the new events introduced in existing ...
1,2016 Winter Youth Olympics,Summary,The 2016 Winter Youth Olympics (Norwegian: Oly...,152,2016 Winter Youth Olympics\nSummary\n\nThe 201...,What was significant about the 2016 Winter You...,The 2016 Winter Youth Olympics held in Lilleha...
2,2016 Winter Youth Olympics,Host selection,Lillehammer was the only city to bid for the g...,129,2016 Winter Youth Olympics\nHost selection\n\n...,What was the process for selecting the host ci...,The process for selecting the host city for th...
3,Olympic Games ceremony,Summary,The Olympic Games ceremonies of the Ancient Ol...,271,Olympic Games ceremony\nSummary\n\nThe Olympic...,What are some elements of the modern Olympic G...,Some elements of the modern Olympic Games cere...
4,Olympic Games ceremony,Ancient forerunners,"The Ancient Games, held in Greece from ca. 776...",247,Olympic Games ceremony\nAncient forerunners\n\...,What were some of the elements involved in the...,Some of the elements involved in the victory c...


In [24]:
len(df)

759

In [25]:
print(df[['context']].values[0][0])

2020 Summer Olympics
Summary

The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and officially branded as Tokyo 2020, were an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina on 7 September 2013.
Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The G

In [27]:
print(df[['questions']].values[0][0])

What were some of the new events introduced in existing sports at the 2020 Summer Olympics in Tokyo?


In [28]:
print(df[['answers']].values[0][0])

Some of the new events introduced in existing sports at the 2020 Summer Olympics in Tokyo were 3x3 basketball, freestyle BMX, mixed gender team events in various sports, the return of madison cycling for men, and the introduction of the same event for women.


#2.4 Save the Olympics Q&A dataset based on Wikipedia sections


In [29]:
df.head()

,title,heading,content,tokens,context,questions,answers
0,2020 Summer Olympics,Summary,"The 2020 Summer Olympics, officially the Games...",623,2020 Summer Olympics\nSummary\n\nThe 2020 Summ...,What were some of the new events introduced in...,Some of the new events introduced in existing ...
1,2016 Winter Youth Olympics,Summary,The 2016 Winter Youth Olympics (Norwegian: Oly...,152,2016 Winter Youth Olympics\nSummary\n\nThe 201...,What was significant about the 2016 Winter You...,The 2016 Winter Youth Olympics held in Lilleha...
2,2016 Winter Youth Olympics,Host selection,Lillehammer was the only city to bid for the g...,129,2016 Winter Youth Olympics\nHost selection\n\n...,What was the process for selecting the host ci...,The process for selecting the host city for th...
3,Olympic Games ceremony,Summary,The Olympic Games ceremonies of the Ancient Ol...,271,Olympic Games ceremony\nSummary\n\nThe Olympic...,What are some elements of the modern Olympic G...,Some elements of the modern Olympic Games cere...
4,Olympic Games ceremony,Ancient forerunners,"The Ancient Games, held in Greece from ca. 776...",247,Olympic Games ceremony\nAncient forerunners\n\...,What were some of the elements involved in the...,Some of the elements involved in the victory c...


In [30]:
df.to_csv('olympics_qa_lg.csv', index=False)

In [31]:
df.head()

,title,heading,content,tokens,context,questions,answers
0,2020 Summer Olympics,Summary,"The 2020 Summer Olympics, officially the Games...",623,2020 Summer Olympics\nSummary\n\nThe 2020 Summ...,What were some of the new events introduced in...,Some of the new events introduced in existing ...
1,2016 Winter Youth Olympics,Summary,The 2016 Winter Youth Olympics (Norwegian: Oly...,152,2016 Winter Youth Olympics\nSummary\n\nThe 201...,What was significant about the 2016 Winter You...,The 2016 Winter Youth Olympics held in Lilleha...
2,2016 Winter Youth Olympics,Host selection,Lillehammer was the only city to bid for the g...,129,2016 Winter Youth Olympics\nHost selection\n\n...,What was the process for selecting the host ci...,The process for selecting the host city for th...
3,Olympic Games ceremony,Summary,The Olympic Games ceremonies of the Ancient Ol...,271,Olympic Games ceremony\nSummary\n\nThe Olympic...,What are some elements of the modern Olympic G...,Some elements of the modern Olympic Games cere...
4,Olympic Games ceremony,Ancient forerunners,"The Ancient Games, held in Greece from ca. 776...",247,Olympic Games ceremony\nAncient forerunners\n\...,What were some of the elements involved in the...,Some of the elements involved in the victory c...


In [32]:
len(df)

759